In [26]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from sympy import *
import os
import moviepy.editor as mp
import imageio
import shutil

# This creates a bifurcation diagram 

The equations are defined as:

In [27]:
values = 100

lambda_vals = np.linspace(0.00001, 0.5, values)
R_vals = np.linspace(0.00001, 5, values)
R = 2
LAMBDA = 1/5
DELTA = 1/6
BIF_VALUE_LAMBDA = 'lambda'
BIF_VALUE_R = 'R'

save_fig = True

def eq_1_solver(lambda_value=LAMBDA, lambda_bool=False, R_value=R, R_bool=False): #f < 0
    
    f = sp.symbols('f')
    if lambda_bool:
        equation = sp.Eq(lambda_value * f, R*DELTA/(DELTA-f) - 1/(1-f))
    if R_bool:
        equation = sp.Eq(LAMBDA * f, R_value*DELTA/(DELTA-f) - 1/(1-f))
    solutions = sp.solve(equation, f)
    real_solutions = [sp.re(sol) for sol in solutions]
    return real_solutions[0], real_solutions[1]

def eq_2_solver(lambda_value=LAMBDA, lambda_bool=False, R_value=R, R_bool=False): #f > 0
    f = sp.symbols('f')
    if lambda_bool:
        equation = sp.Eq(lambda_value * f, R*DELTA/(DELTA+f) - 1/(1+f))
    if R_bool:
        equation = sp.Eq(LAMBDA * f, R_value*DELTA/(DELTA+f) - 1/(1+f))
    solutions = sp.solve(equation, f)
    real_solutions = [sp.re(sol) for sol in solutions]
    return real_solutions[0]

In [28]:
stable_bif_lambda = []
saddle_bif_lambda = []
spiral_bif_lambda = []

for i in lambda_vals:
    stable_bif_lambda.append(eq_1_solver(lambda_value=i, lambda_bool=True)[0])
    saddle_bif_lambda.append(eq_1_solver(lambda_value=i, lambda_bool=True)[1])
    spiral_bif_lambda.append(eq_2_solver(lambda_value=i, lambda_bool=True))


In [29]:
def find_intersection_saddle_and_stable(saddle_bif, stable_bif):
    intersection = []
    intersection_i = []
    for i in range(len(stable_bif)):
        if abs(saddle_bif[i]-stable_bif[i]) < 0.5:
            intersection.append(stable_bif[i])
            intersection_i.append(i)
    return intersection,intersection_i

intersection_lambda, intersection_lambda_i = find_intersection_saddle_and_stable(saddle_bif_lambda, stable_bif_lambda)


In [30]:
def make_bifurcation_diagram(values, stable_bif, saddle_bif, spiral_bif, intersection_i,intersection,k):
    i = len(intersection_i)-1
    j = 1
    plt.plot(values[0:intersection_i[i]+j], stable_bif[0:intersection_i[i]+j], label='Stable node')
    plt.plot(values[0:intersection_i[i]+j], saddle_bif[0:intersection_i[i]+j],label='Saddle', linestyle='dashed')
    plt.plot(values, spiral_bif, label='Stable Spiral')

    if k < intersection_i[i]+j:
        plt.scatter(values[k], stable_bif[k], color='red', marker='o', label='First Saddle')
    else:
        plt.scatter(values[k], spiral_bif[k], color='red', marker='o', label='First Stable Node')

    plt.ylabel(r'equilibrium values of $f$')
    plt.ylim(-4, 1)

    bif_val = r'$\lambda$'
    BIF_VALUE = 'lambda'
    bif_val_int = 0.333800508
    plt.xlabel(f'{bif_val}')
    plt.xlim(0, 0.5)
    plt.xticks([0, 0.1, 0.2, 0.3, bif_val_int, 0.4, 0.5], [0.0, 0.1, 0.2, 0.3, r'$\lambda_b$', 0.4, 0.5])

    x_start = bif_val_int
    y_start = stable_bif[-1]
    x_end = bif_val_int
    y_end = spiral_bif[-1]

    arrow_props = dict(arrowstyle='->', color='k', lw=1, ls='dashed')
    plt.annotate('', xy=(x_end, y_end+0.07), xytext=(x_start, intersection[-1]),
             arrowprops=arrow_props, fontsize=9, color='k')

    #plt.legend()
    folder_name = 'Plots for bifurcation diagram'
    folder_path = os.path.join(os.getcwd(), folder_name)
    
    # Ensure the folder exists, create it if it doesn't
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    file_name = f'bifurcation_diagram_{BIF_VALUE}_{k}.png'
    desktop_path = os.path.join(folder_path, file_name)

    if save_fig:
        plt.savefig(desktop_path, dpi=600)



In [31]:
for i in range(10, len(lambda_vals)):
    make_bifurcation_diagram(lambda_vals, 
                            stable_bif_lambda, 
                            saddle_bif_lambda, 
                            spiral_bif_lambda, 
                            intersection_lambda_i,
                            intersection_lambda, 
                            i)
    plt.clf()


<Figure size 432x288 with 0 Axes>

# Gif creator

In [32]:
def create_gif(image_folder, output_file):
    images = []
    filenames = []
    for filename in os.listdir(image_folder):
        filenames.append(filename)
    filenames = sorted(filenames)

    for filename in filenames:
        try:
            images.append(imageio.imread(os.path.join(image_folder, filename)))
        except:
            print(f'Error in {filename}')
            pass
    imageio.mimsave(output_file, images, duration=0.15)

gif_folder_name = 'Gifs for bifurcation diagram'
movie_folder_name = 'Movies for bifurcation diagram'

if os.path.exists(gif_folder_name):
    shutil.rmtree(gif_folder_name)
    os.mkdir(gif_folder_name)
else:
    os.mkdir(gif_folder_name)

if os.path.exists(movie_folder_name):
    shutil.rmtree(movie_folder_name)
    os.mkdir(gif_folder_name)
else:
    os.mkdir(movie_folder_name)

create_gif('Plots for bifurcation diagram', gif_folder_name + '/bifurcation_diagram.gif')

clip = mp.VideoFileClip(gif_folder_name+"/bifurcation_diagram.gif")

clip.write_videofile(movie_folder_name+"/bifurcation_diagram.mp4")


Moviepy - Building video Movies for bifurcation diagram/bifurcation_diagram.mp4.
Moviepy - Writing video Movies for bifurcation diagram/bifurcation_diagram.mp4



Moviepy - Done !
Moviepy - video ready Movies for bifurcation diagram/bifurcation_diagram.mp4
